Proyecto Final de Trading

Iteso 

Arturo Ruiz

Juan Carlos Gonzalez



In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import datetime

In [ ]:
#Laboratorio 4 continuación
def get_prices(date):
    A1_OA_Da = 16                     # Day Align
    A1_OA_Ta = "America/Mexico_City"  # Time Align
    A1_OA_At = "practice"             # Tipo de cuenta
    A1_OA_In = "USD_MXN"              # Instrumento
    A1_OA_Gn = "M5"                   # Granularidad de velas
    A1_OA_Ak = "ee37e0cfccd3a6d2f842f120aaab81a4-3d14f03cc84eb3fd3fcb14ec3f999fec"
    ##obtener el spot
    a=datetime.datetime.strptime(date,"%m/%d/%Y %H:%M")
    today = a
    DD = datetime.timedelta(minutes=55)
    earlier = today - DD
    later = today + DD
    F1=later.strftime("%Y-%m-%dT%H:%M:%SZ")
    F2=earlier.strftime("%Y-%m-%dT%H:%M:%SZ")
    api = API(access_token=A1_OA_Ak)
    params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
              "alignmentTimezone": A1_OA_Ta, "from": F2, "to": F1}
    A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
    A1_Hist = api.request(A1_Req1)
    a=list(map(lambda i:[A1_Hist['candles'][i]['time'],float(A1_Hist['candles'][i]['mid']['o']),float(A1_Hist['candles'][i]['mid']['h']),
                          float(A1_Hist['candles'][i]['mid']['l']),float(A1_Hist['candles'][i]['mid']['c'])],range(len(A1_Hist['candles']))))
    pd_hist = pd.DataFrame(a,columns=['TimeStamp','Open','High','Low','Close'])
    pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
    pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])
    
    return pd_hist 

In [ ]:
data=pd.read_csv('data.csv')

In [ ]:
a=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]]
b=data.loc[[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]]
c=data.loc[[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]]
d=data.loc[[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]]
data['a']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus>=data.Previous)]
data['b']=[a and b for a,b in zip(data.Actual>=data.Consensus,data.Consensus<data.Previous)]
data['c']=[a and b for a,b in zip(data.Actual<data.Consensus,data.Consensus>=data.Previous)]
data['d']=[a and b for a,b in zip(data.Actual<=data.Consensus,data.Consensus<data.Previous)]


In [ ]:
# direcciones de los movimientos

In [ ]:
week_days=list(map(lambda x:datetime.datetime.weekday(datetime.datetime.strptime(data.DateTime[x],"%m/%d/%Y %H:%M")),range(len(data))))

In [ ]:
temp=list(map(lambda x:get_prices(data.DateTime[x]),range(len(data))))

In [ ]:
c1_direction=list(map(lambda x:temp[x].loc[21].Close-temp[x].loc[11].Open,range(len(temp))))

In [ ]:
c2_vol=list(map(lambda x:max(temp[x].High)-min(temp[x].Low),range(len(temp))))

In [ ]:
c3_drawDown=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].Low,range(len(temp))))

In [ ]:
c4_drawUp=list(map(lambda x:temp[x].loc[0].Open-temp[x].loc[21].High,range(len(temp))))

In [ ]:
#direcciones 
data['c1_direction']=c1_direction
data['c2_vol']=c2_vol
data['c3_drawDown']=c3_drawDown
data['c4_drawUp']=c4_drawUp


In [ ]:
#frecuancia de cada escenario
po=[sum(data['a'])/len(data),
sum(data['b'])/len(data),
sum(data['c'])/len(data),
sum(data['d'])/len(data)]


In [ ]:
tp=np.array([sum(data.loc[a.index]['c1_direction']>0)/sum(data['a']),
sum(data.loc[b.index]['c1_direction']>0)/sum(data['b']),
sum(data.loc[c.index]['c1_direction']>0)/sum(data['c']),
sum(data.loc[d.index]['c1_direction']>0)/sum(data['d'])])

In [ ]:
tn=1-tp

In [ ]:
results=np.transpose(pd.DataFrame([tp,tn,po],columns=['a','b','c','d'],index=['direccion positiva','direccion negativa','porcentaje ocurrencia']))

In [ ]:
#los 4 escenarios finales
results

Los parámetros a optimizar son: take profit, stop loss, la ventana de tiempo para hacer la compra, la cantidad de títulos más optima es realizar la compra y venta con el total de titulos posibles, y el numero de eventos que se utilizaran a c  d, o solamente 1 o 2 o 3.

In [ ]:
total=10000 
trade=[]
stop_loss=5 # de 1 a 1000
take_profit=5 # de 1 a 1000
momento_compra=11 # momento en que sale la noticia y eventos posteriores de 0 a 20
n_eventos=1 #de 1 a 0


# Esta parte se encarga de decirte en que posicion queda, si es que hay un cumplimiento del stop loss o take profit.
for i in range(len(temp)):
    
    if (results.loc[data.iloc[i,7:11]]['direccion positiva']<=n_eventos)[0] and (results.loc[data.iloc[i,7:11]]['direccion positiva']>0)[0]:

        #en temp estan todos los registros de datos almacenados de oanda
        dolares_totales=np.trunc(total/temp[i].loc[0].Open)
        monto_compra=dolares_totales*temp[i].loc[momento_compra].Open
        total-=monto_compra
        
        if True in list(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)<-stop_loss/10000):
            stop_loss_position=np.where(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)<-stop_loss/10000)[0][0]+momento_compra+1
            stop_loss_profit=dolares_totales*temp[i].loc[stop_loss_position].Close
        else:
            stop_loss_position=None
        if True in list(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)>take_profit/10000):
            take_profit_position=np.where(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)>take_profit/10000)[0][0]+momento_compra+1
            take_profit_profit=dolares_totales*temp[i].loc[take_profit_position].Close
        else:
            take_profit_position=None
        
        # En esta parte se evalua la parte de arriba, si sucedió el stop loss o take profit y en cual de los 4 casos posibles es el que sucedió.
        
        if not stop_loss_position==None and take_profit_position==None:
            monto_venta=stop_loss_profit
        elif not take_profit_position==None and stop_loss_position==None:
            monto_venta=take_profit_profit
        elif not take_profit_position==None and not stop_loss_position==None:
            if take_profit_position<stop_loss_position:
                monto_venta=take_profit_profit
            else:
                monto_venta=stop_loss_profit
        else:
            monto_venta=dolares_totales*temp[i].loc[21].Close
        
        
        pL=monto_venta-monto_compra
        total+=monto_venta
        trade.append([dolares_totales,pL,total])
    
    else:
        trade.append([0,0,total])
# esto nos devuelve una lista de como se realizaron todos los trades de la cuenta y el balance final de esta

In [ ]:
# en esta parte comienza la optimización de los parámetros
from scipy.optimize import Bounds
from scipy.optimize import minimize

In [ ]:
# En esta parte del código se tiene todo lo de arriba pero hecha función para llamar los parámetros a optimizar.

def trading(stop_loss,take_profit,momento_compra,n_eventos):
    total=10000 
    trade=[]
    
    # En este for recorre todos los eventos y checa en que estado estan a,b,c o d.
    for i in range(len(temp)):
        if (results.loc[data.iloc[i,7:11]]['direccion positiva']<=n_eventos)[0] and (results.loc[data.iloc[i,7:11]]['direccion positiva']>0)[0]:

            dolares_totales=np.trunc(total/temp[i].loc[0].Open)
            monto_compra=dolares_totales*temp[i].loc[momento_compra].Open
            total-=monto_compra

            if True in list(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)<-stop_loss/10000):
                stop_loss_position=np.where(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)<-stop_loss/10000)[0][0]+momento_compra+1
                stop_loss_profit=dolares_totales*temp[i].loc[stop_loss_position].Close
            else:
                stop_loss_position=None
            if True in list(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)>take_profit/10000):
                take_profit_position=np.where(np.array(temp[i].iloc[momento_compra+1:,-1])-np.array(temp[i].loc[momento_compra].Open)>take_profit/10000)[0][0]+momento_compra+1
                take_profit_profit=dolares_totales*temp[i].loc[take_profit_position].Close
            else:
                take_profit_position=None



            if not stop_loss_position==None and take_profit_position==None:
                monto_venta=stop_loss_profit
            elif not take_profit_position==None and stop_loss_position==None:
                monto_venta=take_profit_profit
            elif not take_profit_position==None and not stop_loss_position==None:
                if take_profit_position<stop_loss_position:
                    monto_venta=take_profit_profit
                else:
                    monto_venta=stop_loss_profit
            else:
                monto_venta=dolares_totales*temp[i].loc[21].Close


            pL=monto_venta-monto_compra
            total+=monto_venta
            trade.append([dolares_totales,pL,total])

        else:
            trade.append([0,0,total])
    
    trades=pd.DataFrame(trade,columns=['total quantity','P&L','Total'])
    return trades.iloc[-1]['Total']

# la función te regresa el total de lo que se generó en la cuenta ya con los parámetros optimizados(falta el pso)

In [ ]:
trading(500,500,11,1)

In [ ]:
tr